## Ordenación eficaz de datos con Amazon DynamoDB

Amazon DynamoDB ofrece alta escalabilidad y desempeño para aplicaciones con cargas de trabajo variables. Aunque DynamoDB destaca en la distribución eficaz de datos en varias particiones, sigue de forma inherente un orden de clasificación específico basado en el esquema seleccionado. En esta publicación, mostramos dos modelos de datos de ejemplo, uno diseñado para almacenar información de pedidos de comercio electrónico y otro para almacenar resultados de partidos. Utilizamos estos modelos de datos para explorar cómo DynamoDB ordena los elementos de forma natural y profundizar en estrategias eficaces para establecer un orden personalizado.

Antes de examinar los detalles, es esencial comprender la importancia de las claves de partición y ordenación en DynamoDB y cómo podemos aprovechar sus ventajas para crear un modelo de datos eficaz y escalable.

### Características de la clave de partición y la clave de ordenación

Una clave principal compuesta en DynamoDB consta de dos atributos: la clave de partición y la clave de ordenación. El valor de la clave de partición se utiliza como entrada para una función hash interna, que determina la partición específica (almacenamiento físico interno dentro de DynamoDB) en la que se almacena el elemento. Los elementos con el mismo valor de clave de partición se almacenan juntos y se ordenan en función de sus valores de clave de ordenación.

En las tablas con una clave de partición y una clave de ordenación, es posible que varios elementos compartan el mismo valor de clave de partición. Sin embargo, los elementos con la misma clave de partición deben tener valores de clave de ordenación distintos.

### Ordenación

La clave de ordenación, también conocida como clave de rango, es la responsable de determinar el orden en el que se almacenan los elementos dentro de una partición. Cuando consulta o explora una tabla de DynamoDB, la clave de ordenación le permite recuperar datos en un orden específico basado en los valores de la clave de ordenación.

Los elementos que comparten el mismo valor de clave de partición se organizan en función de la clave de ordenación. El mecanismo de ordenación varía en función del tipo de datos de la clave de ordenación:

Si el tipo de datos de la clave de ordenación es Número, DynamoDB organiza los elementos en orden numérico, garantizando que las comparaciones numéricas sean directas y eficientes.
Cuando la clave de ordenación es de tipo Cadena, DynamoDB secuencia los elementos de acuerdo con el orden de bytes UTF-8, lo que resulta ideal para la ordenación lexicográfica.
Para los tipos de datos Binarios, DynamoDB trata cada byte de los datos binarios como sin signo, facilitando una ordenación precisa a nivel de byte.
Condiciones

La clave de ordenación de una tabla de DynamoDB es una potente herramienta para optimizar la eficacia de las consultas. Combinando la clave de ordenación con condiciones, puede realizar consultas precisas y eficientes que recuperen sólo los datos que necesita. Por ejemplo, puede utilizar condiciones para recuperar elementos con un rango de atributos ordenable, como una fecha. Esto permite una recuperación específica, reduciendo la cantidad de datos escaneados y mejorando el rendimiento de la consulta. Si diseña estratégicamente su modelo de datos y aprovecha eficazmente la clave de ordenación, podrá adaptar sus consultas para que se ajusten a distintos patrones de acceso y acceder de forma eficaz a los datos más importantes para su aplicación.

### Ejemplo de modelo de datos de comercio electrónico

Para comprender mejor cómo funciona la ordenación en relación con la clave de partición, visualicemos el concepto. DynamoDB almacena los datos en elementos (análogos a las filas) donde cada elemento tiene un identificador único denominado clave de partición, que sirve como forma principal de distribuir los datos entre las particiones. Este modelo utiliza una clave de ordenación, que determina el orden de los elementos dentro de cada partición. Nuestra tabla de DynamoDB contiene pedidos de usuarios, con el ID de usuario como clave de partición y su fecha de pedido como clave de ordenación. DynamoDB no tiene un tipo de datos de fecha nativo, por lo que nuestra clave de ordenación utiliza un formato de cadena ISO8601.

Para la clave de partición userID, DynamoDB distribuye los datos de los usuarios entre las particiones en función de su ID. Dentro de cada partición, DynamoDB ordena los datos por la clave de ordenación, que es la fecha del pedido. Visualizando esto, podemos pensar en los datos organizados como en un archivador:

* Cada cajón del archivador representa una partición, identificada por un ID de usuario único.
* Dentro de cada cajón (partición), se encuentran los archivos (elementos) de cada usuario, ordenados por su fecha de pedido.

La siguiente tabla ilustra nuestro caso de uso de ejemplo. Podemos referirnos a un grupo de elementos que comparten la misma clave de partición (pero diferentes claves de ordenación) como una colección de elementos.

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import base64
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [2]:
dt = DynamoTable(
    table_name='OrdersTable',
    region_name='us-west-2',
    #profile_name='my-profile'
)

if not dt.table_name:
    dt.create_table(
        table_name='OrdersTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
        region_name='us-west-2',
        #profile_name='my-profile'
    )
    
else:
    print(dt)

- Table name: OrdersTable            
- Table arn: arn:aws:dynamodb:us-west-2:089715336747:table/OrdersTable            
- Table creation: 2024-01-12 12:15:34            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'GSI1PK', 'AttributeType': 'S'}, {'AttributeName': 'GSI2PK', 'AttributeType': 'S'}, {'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'SK', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [33]:
data = [
    {'PK': 'USER#user923', 'SK': '2023-10-03T12:32:000Z', 'order_id': 'ORDER#order001', 'product': 'Hammer', 'price': 3.99, 'GSI1PK': '2023-10-03', 'GSI2PK': '1'}, 
    {'PK': 'USER#user923', 'SK': '2023-10-06T01:44:000Z', 'order_id': 'ORDER#order015', 'product': 'Caulk', 'price': 4.59, 'GSI1PK': '2023-10-06', 'GSI2PK': '1'}, 
    {'PK': 'USER#user923', 'SK': '2023-10-12T17:23:000Z', 'order_id': 'ORDER#order023', 'product': 'Paint', 'price': 14.35, 'GSI1PK': '2023-10-12', 'GSI2PK': '1'}, 
    {'PK': 'USER#user226', 'SK': '2023-10-04T01:45:000Z', 'order_id': 'ORDER#order011', 'product': 'Floor Brush', 'price': 19.00, 'GSI1PK': '2023-10-04', 'GSI2PK': '1'}, 
    {'PK': 'USER#user845', 'SK': '2023-10-03T14:22:000Z', 'order_id': 'ORDER#order004', 'product': 'Nails', 'price': 3.58, 'GSI1PK': '2023-10-03', 'GSI2PK': '1'}, 
    {'PK': 'USER#user845', 'SK': '2023-10-19T13:02:000Z', 'order_id': 'ORDER#order030', 'product': 'Hammer', 'price': 3.99, 'GSI1PK': '2023-10-19', 'GSI2PK': '1'}, 
    {'PK': 'USER#user575', 'SK': '2023-10-21T13:02:000Z', 'order_id': 'ORDER#order099', 'product': 'Phone', 'price': 88.35, 'GSI1PK': '2023-10-21', 'GSI2PK': '1'}, 
    {'PK': 'USER#user575', 'SK': '2023-10-22T13:02:000Z', 'order_id': 'ORDER#order100', 'product': 'Laptop', 'price': 999.99, 'GSI1PK': '2023-10-22', 'GSI2PK': '1'}, 
    {'PK': 'USER#user575', 'SK': '2023-10-23T13:02:000Z', 'order_id': 'ORDER#order101', 'product': 'Tablet', 'price': 799.99, 'GSI1PK': '2023-10-23', 'GSI2PK': '1'}
]
df = pd.DataFrame(columns=['PK', 'SK', 'order_id', 'product', 'price', 'GSI1PK', 'GSI2PK'], data=data)

In [34]:
dt.batch_pandas(df)

In [4]:
dt.query(
    pk_value='USER#user923', 
    sk_value='2023-10-06*', 
    to_pandas=True
)

,product,GSI2PK,GSI1PK,SK,order_id,price,PK
0,Caulk,1,2023-10-06,2023-10-06T01:44:000Z,ORDER#order015,4.59,USER#user923


Ahora imaginemos que las necesidades de negocio adicionales tienen los requisitos añadidos para estos patrones de acceso adicionales:

* Obtener todos los pedidos de las últimas 24 horas
* Obtener todos los pedidos de los últimos 7 días
* Obtener todos los pedidos de los últimos 1 mes
* Obtener todos los pedidos de los últimos 3 meses

Hemos visto cómo DynamoDB mantiene el orden dentro de una colección de elementos según el valor de la clave de ordenación. Estos nuevos patrones de acceso requieren un orden que abarque todos los elementos (o todas las claves de partición).

#### Resumen de la solución

Para establecer un orden que abarque todas las claves de partición, una observación crítica es que carecemos de un atributo que nos permita agrupar los datos en una colección de elementos unificada.

Si la recuperación de todos los pedidos del pasado no es un patrón de acceso comúnmente solicitado, podemos utilizar una operación de exploración y filtrar los resultados para que coincidan con el marco temporal deseado. Sin embargo, este método puede resultar ineficaz tanto en términos de rendimiento como de coste. Por lo tanto, si este patrón de acceso se solicita con frecuencia, necesitaremos un enfoque alternativo.

#### Aprovechamiento de un índice secundario global

Un índice secundario global (GSI) es una función de DynamoDB que mantiene una copia coherente de algunos o todos los datos de la tabla base. Los índices secundarios globales permiten realizar consultas eficientes en una tabla basándose en atributos distintos de la clave principal. Proporcionan flexibilidad para consultar y filtrar datos, admiten consultas paralelas y son esenciales para optimizar el rendimiento de las consultas al tiempo que se adaptan a diversos patrones de acceso.

Ahora que entendemos cómo DynamoDB mantiene el orden dentro de una colección de elementos, podemos diseñar un esquema alternativo para soportar nuestros patrones de acceso adicionales utilizando un GSI.

#### Enfoque 1 (no óptimo)

Reconociendo la capacidad de las colecciones de elementos para organizar los datos de forma eficaz, hemos implementado un Índice Secundario Global (GSI) que utiliza un atributo de fecha con una granularidad de un día. Esto nos permite agrupar eficazmente los pedidos de cada día concreto. Para facilitarlo, hemos introducido un atributo adicional en nuestra estructura de datos denominado gsi1_pk que almacena los valores de fecha necesarios.

In [15]:
dt.create_global_secondary_index(
    att_name="GSI1PK",
    att_type="S",
    i_name="GSI1"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("Global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        time.sleep(30)
    end = time.time()
    minute = (end - start) / 60
    print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

Global secondary index is being created, this may take a few minutes...
Global secondary index created. Time elapsed: 6.05 minute


Ahora, tenemos la capacidad de consultar datos de forma eficiente basándonos en días específicos, como recuperar todos los pedidos de 2023-10-03. Aunque este enfoque es eficaz para consultas de un solo día, nuestro caso de uso exige manejar intervalos de fechas más amplios. Por ejemplo, para obtener los datos de toda una semana sería necesario realizar siete peticiones paralelas, una por cada día de la semana. Aunque es manejable para una semana, es importante tener en cuenta que a medida que se amplía el intervalo de fechas, el número de peticiones necesarias aumenta linealmente, lo que puede afectar a la escalabilidad.

In [4]:
dt.query(
    pk_value = "2023-10-03",
    index_name = "GSI1",
    to_pandas = True,
    consumed_capacity = "TOTAL"
)

Consumed Capacity: 0.5


,GSI2PK,product,GSI1PK,SK,order_id,price,PK
0,1,Hammer,2023-10-03,2023-10-03T12:32:000Z,ORDER#order001,3.99,USER#user923
1,1,Nails,2023-10-03,2023-10-03T14:22:000Z,ORDER#order004,3.58,USER#user845


In [5]:
dt.query(
    pk_value = "2023-10-04",
    index_name = "GSI1",
    to_pandas = True,
    consumed_capacity = "TOTAL"
)

Consumed Capacity: 0.5


,GSI2PK,product,GSI1PK,SK,order_id,price,PK
0,1,Floor Brush,2023-10-04,2023-10-04T01:45:000Z,ORDER#order011,19,USER#user226


In [6]:
dt.query(
    pk_value = "2023-10-06",
    index_name = "GSI1",
    to_pandas = True,
    consumed_capacity = "TOTAL"
)

Consumed Capacity: 0.5


,GSI2PK,product,GSI1PK,SK,order_id,price,PK
0,1,Caulk,2023-10-06,2023-10-06T01:44:000Z,ORDER#order015,4.59,USER#user923


### Enfoque 2 (óptimo)

Una estrategia mejorada consiste en aprovechar la clave de ordenación, lo que nos permite utilizar las condiciones de forma eficaz. En este enfoque, seleccionamos un valor fijo para nuestra clave de partición GSI gsi1_pk, consolidando de hecho todos los datos en una única colección de elementos. La clave de clasificación se define como una marca de tiempo ISO 8601 (cadena) con una granularidad de milisegundos. Estas marcas de tiempo ya están almacenadas en nuestros artículos bajo el atributo SK.

In [32]:
dt.create_global_secondary_index(
    att_name="GSI2PK",
    att_type="S",
    sort_index="SK",
    sort_type="S",
    i_name="GSI2"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("Global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        time.sleep(30)
    end = time.time()
    minute = (end - start) / 60
    print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

Global secondary index is being created, this may take a few minutes...
Global secondary index created. Time elapsed: 6.55 minute


In [6]:
dt.query(
    pk_value = "1",
    sk_value = "2023-10-03_2023-10-06",
    index_name = "GSI2",
    to_pandas = True,
    consumed_capacity = "TOTAL"
)

Consumed Capacity: 0.5


,GSI2PK,product,GSI1PK,SK,order_id,price,PK
0,1,Hammer,2023-10-03,2023-10-03T12:32:000Z,ORDER#order001,3.99,USER#user923
1,1,Nails,2023-10-03,2023-10-03T14:22:000Z,ORDER#order004,3.58,USER#user845
2,1,Floor Brush,2023-10-04,2023-10-04T01:45:000Z,ORDER#order011,19.00,USER#user226


In [4]:
dt.query(
    pk_value = "1",
    sk_value = "2023-10-03_2023-10-06",
    index_name = "GSI2",
    to_pandas = True,
    consumed_capacity = "TOTAL"
)

Consumed Capacity: 0.5


,GSI2PK,product,GSI1PK,SK,order_id,price,PK
0,1,Hammer,2023-10-03,2023-10-03T12:32:000Z,ORDER#order001,3.99,USER#user923
1,1,Nails,2023-10-03,2023-10-03T14:22:000Z,ORDER#order004,3.58,USER#user845
2,1,Floor Brush,2023-10-04,2023-10-04T01:45:000Z,ORDER#order011,19.00,USER#user226


Este enfoque no sólo mejora la eficiencia, sino que también ofrece una mayor flexibilidad. Si nuestras necesidades de negocio evolucionan para abarcar patrones de acceso alternativos, como recuperar todos los pedidos de los últimos 30 minutos, obtener los 100 pedidos más recientes o acceder a los 100 pedidos más antiguos, nuestro modelo de datos nos proporciona la versatilidad necesaria para ejecutar estas consultas de forma eficiente.

El uso de un único valor fijo como clave de partición puede introducir cuellos de botella en el rendimiento, que abordaremos más adelante en este post.

### Modelo de datos de juegos

DynamoDB se utiliza con frecuencia para almacenar información de juegos, como resultados e información sobre los jugadores. Sus capacidades de escalabilidad y rendimiento la convierten en una buena opción para las aplicaciones de juegos. El diseño flexible del esquema permite realizar ajustes en la mecánica del juego sin complicadas modificaciones en la base de datos, mientras que sus operaciones de baja latencia garantizan actualizaciones en tiempo real, lo que la convierte en una opción fiable para gestionar tablas de clasificación, perfiles de jugadores y logros en aplicaciones de juegos. Un modelo de datos de ejemplo puede tener el siguiente aspecto:

In [2]:
dt = DynamoTable(
    table_name='GameTable',
    region_name='us-west-2'
)

if not dt.table_name:
    dt.create_table(
        table_name='GameTable',
        partition_key='PK',
        partition_key_type='S',
        region_name='us-west-2'
    )
    
else:
    print(dt)

- Table name: GameTable            
- Table arn: arn:aws:dynamodb:us-west-2:089715336747:table/GameTable            
- Table creation: 2024-01-12 22:52:53            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}]            
- [{'AttributeName': 'GSI1PK', 'AttributeType': 'S'}, {'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'score', 'AttributeType': 'N'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [7]:
data = [
    {'PK': 'user920', 'score': 7, 'f_name': 'Nikki', 's_name': 'Wolf', 'creation_date': '2023-10-03T12:32:000Z', 'GSI1PK': '1'},
    {'PK': 'user925', 'score': 9, 'f_name': 'Marvin', 's_name': 'Deam', 'creation_date': '2023-10-03T15:12:000Z', 'GSI1PK': '1'},
    {'PK': 'user928', 'score': 3, 'f_name': 'Linda', 's_name': 'Barnes', 'creation_date': '2023-10-04T17:32:000Z', 'GSI1PK': '1'},
    {'PK': 'user855', 'score': 5, 'f_name': 'John', 's_name': 'Smith', 'creation_date': '2023-10-05T18:32:000Z', 'GSI1PK': '1'}
]
df = pd.DataFrame(columns=['PK', 'score', 'f_name', 's_name', 'creation_date', 'GSI1PK'], data=data)
dt.batch_pandas(df)

En este ejemplo de modelo de datos, observamos una clave primaria simple definida como una clave de partición que representa el identificador único del usuario, denominado userId. Este diseño resulta muy eficaz para consultas clave-valor sencillas centradas en el userId, como la recuperación de la puntuación del usuario0011 o la actualización de la puntuación del usuario30046.

Imaginemos un nuevo caso de uso para generar tablas de clasificación que muestren los 10 y 50 mejores usuarios de nuestro juego. Aunque puede parecer intuitivo introducir un atributo "puntuación" como clave de ordenación para facilitar este patrón de acceso, este enfoque se enfrenta a dos retos importantes que lo hacen poco práctico. En DynamoDB, no se pueden modificar las claves primarias de la tabla principal, lo que impide realizar actualizaciones eficientes del valor de puntuación. Por ejemplo, no puede utilizar UpdateItem para cambiar los atributos de clave primaria. En su lugar, debe eliminar el elemento y, a continuación, utilizar PutItem para introducir un nuevo elemento con los atributos deseados. Y lo que es más importante, DynamoDB limita la clasificación a la colección de elementos, lo que en este contexto significa que el elemento de cada usuario constituiría una colección de elementos que comprendería un único elemento, lo que socavaría la viabilidad de la funcionalidad de clasificación deseada.

Podemos utilizar una solución similar a la de nuestra tabla de pedidos de comercio electrónico, crear un índice secundario global con un valor de clave de partición estático, de modo que todos los elementos de usuario se mantengan dentro de una única colección de elementos, y utilizar el atributo de puntuación como clave de ordenación del índice secundario global. Así que incluimos gsi1_pk como un atributo en nuestro modelo de datos:

In [8]:
dt.create_global_secondary_index(
    att_name="GSI1PK",
    att_type="S",
    sort_index="score",
    sort_type="N",
    i_name="GSI1"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("Global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        time.sleep(30)
    end = time.time()
    minute = (end - start) / 60
    print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

Global secondary index is being created, this may take a few minutes...
Global secondary index created. Time elapsed: 3.02 minute


In [3]:
dt.query(
    pk_value = "1",
    index_name = "GSI1",
    to_pandas = True,
    consumed_capacity = "TOTAL",
    reverse = False
)

Consumed Capacity: 0.5


,GSI1PK,f_name,s_name,score,PK,creation_date
0,1,Marvin,Deam,9,user925,2023-10-03T15:12:000Z
1,1,Nikki,Wolf,7,user920,2023-10-03T12:32:000Z
2,1,John,Smith,5,user855,2023-10-05T18:32:000Z
3,1,Linda,Barnes,3,user928,2023-10-04T17:32:000Z


Esta solución nos permite recuperar las N puntuaciones más altas de nuestro índice de forma eficiente, minimizando cualquier consumo innecesario de capacidad de lectura.

### Consideraciones sobre costes

Al utilizar un índice secundario global (GSI) en DynamoDB, factores como la proyección de atributos, el almacenamiento y el rendimiento afectan a los costes. La elección de la proyección de atributos es una consideración crucial. El uso de la proyección INCLUDE permite seleccionar un subconjunto de atributos que se incluirán en el índice, reduciendo así los costes de almacenamiento. Por el contrario, la proyección ALL incluye todos los atributos en el índice, lo que simplifica la consulta pero aumenta significativamente los costes de almacenamiento y rendimiento. Lograr un equilibrio entre la optimización de costes y la funcionalidad es vital, y requiere una evaluación cuidadosa de la importancia de los atributos proyectados, la frecuencia de su acceso y el presupuesto disponible. Sopesando estos factores, se pueden tomar decisiones informadas para optimizar los costes manteniendo el nivel deseado de rendimiento y funcionalidad.

### Consideraciones sobre el rendimiento

Aunque adoptar un único valor como clave de partición en el modelo de datos de DynamoDB es conveniente, implica concesiones que pueden afectar al rendimiento. Cuando se utiliza un valor estático como clave de partición para un índice secundario global, todos los elementos de datos se concentran en una única partición. Como resultado, la carga de trabajo no se distribuirá uniformemente entre varias particiones, lo que limita las ventajas del escalado horizontal en DynamoDB.

Esta concentración de datos en una única partición puede provocar cuellos de botella en el rendimiento. En escenarios en los que se esperan altas tasas de lectura o escritura, este enfoque puede dar lugar a particiones calientes, en las que una única partición se satura con un exceso de solicitudes de lectura o escritura. Esto puede dar lugar a un estrangulamiento, un aumento de la latencia y una reducción del rendimiento general. Por lo tanto, teniendo en cuenta los límites de partición de DynamoDB, esta estrategia es más adecuada para tablas en las que el tráfico de escritura no supere las 1000 WCU y el tráfico de lectura de índices se mantenga por debajo de las 3000 RCU.

Se debe considerar cuidadosamente la selección de una estrategia de clave de partición adecuada para garantizar un rendimiento y una escalabilidad óptimos en DynamoDB.

### Optimización del rendimiento mediante la fragmentación de claves de partición

Para abordar los problemas de escalabilidad asociados con un único valor como clave de partición, puede emplear la fragmentación de claves de partición GSI. Este enfoque le permite distribuir uniformemente la carga de trabajo entre varias particiones, lo que mejora el rendimiento y la escalabilidad.

La fragmentación de claves de partición implica dividir los datos entre múltiples claves de partición en lugar de confiar en un único valor estático como clave de partición. Este enfoque distribuye la carga de trabajo entre varias particiones, aprovechando el escalado horizontal de DynamoDB.

La fragmentación de claves de partición ofrece varias ventajas:

* **Escalabilidad mejorada** - Al distribuir la carga de trabajo entre varias particiones, la fragmentación de claves de partición permite una mejor escalabilidad a medida que crece la aplicación.

* **Rendimiento mejorado** - Con la carga de trabajo distribuida uniformemente, la fragmentación de claves de partición ayuda a evitar particiones calientes, evitando cuellos de botella en el rendimiento y reduciendo la probabilidad de estrangulamiento.

* **Flexibilidad en la distribución de datos** - La fragmentación por claves de partición proporciona la flexibilidad necesaria para ajustar y optimizar la distribución de datos en función de los cambios en los patrones de acceso y de los requisitos cambiantes de la aplicación.

La fragmentación por rangos calculados puede ser la solución más sencilla para particionar los datos. En este enfoque, se asignan valores que van de 1 a N como claves de partición. Con cada partición de DynamoDB capaz de proporcionar un rendimiento máximo de 1000 WCUs por segundo, puede determinar el valor adecuado para N mediante la siguiente fórmula:

>> N = rendimiento_pico_esperado / 1000

Al dividir el rendimiento máximo esperado por 1000, puede calcular el número de particiones (N) necesarias para distribuir la carga de trabajo uniformemente por la tabla de DynamoDB. Esto garantiza que cada partición reciba una cantidad manejable de solicitudes y evita la sobrecarga de una sola partición. La fragmentación de rango calculada simplifica el proceso de determinar el número de particiones necesarias para lograr el rendimiento y la escalabilidad deseados en DynamoDB.

Cuando se utiliza la fragmentación de escritura GSI en DynamoDB, el proceso de recuperación de datos se vuelve más complejo, ya que requiere la ejecución de solicitudes paralelas en las distintas particiones. La fragmentación de escritura GSI implica la distribución de datos en varias particiones en función de la clave de partición elegida, incluido el sufijo de partición. Cada partición gestiona un subconjunto de los datos, lo que permite mejorar la escalabilidad y el rendimiento. Sin embargo, cuando se obtienen datos, es necesario consultar todas las particiones relevantes simultáneamente para obtener un conjunto de resultados completo. Para ello, es necesario realizar peticiones paralelas a cada partición y combinar los resultados para obtener una visión completa y coherente de los datos. Al aprovechar el paralelismo de esta manera, DynamoDB garantiza una recuperación eficaz de los datos en varias particiones, lo que permite operaciones de lectura concurrentes y de alta velocidad de forma distribuida y escalable.

### Conclusión

En este artículo, hemos demostrado la utilización de un Índice Secundario Global (GSI) en combinación con el ordenamiento por clave para obtener un orden global en todos los elementos. Este enfoque nos permite organizar y acceder a nuestros datos de forma eficaz, lo que garantiza una base sólida para diversas necesidades de consulta y un rendimiento mejorado en DynamoDB.

Además, la fragmentación de claves de partición proporciona una solución para superar las solicitudes no distribuidas asociadas con el uso de un único valor como clave de partición en DynamoDB. Al distribuir la carga de trabajo entre varias particiones, esta técnica mejora la escalabilidad y el rendimiento. Al adoptar la fragmentación de claves de partición en su modelo de datos de DynamoDB, puede lograr un mejor rendimiento, escalabilidad y flexibilidad, garantizando una utilización óptima de los recursos de la base de datos.

Dé el siguiente paso para dominar las capacidades de modelado de datos de DynamoDB y optimizar el rendimiento de su base de datos. Explore la sección de modelado de datos de la documentación para obtener información detallada y aprenda a crear y gestionar índices secundarios globales (GSI) para mejorar aún más sus conocimientos sobre DynamoDB.